In [4]:
import folium
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from pyproj import Proj
from collections import defaultdict
from coord_convert.transform import gcj2wgs, wgs2gcj
from shapely.geometry import Polygon, Point
from shapely.ops import unary_union
from shapely.affinity import scale
from pprint import pprint
projector = Proj(f"+proj=tmerc +lat_0=39.766541 +lon_0=116.552786")

In [7]:
# 可视化路区, 找一个合适的中心点(大致即可), 作为constants_spatial.py中的LON_CEN, LAT_CEN
regions = pd.read_csv("orig_data/region.csv").values

m = folium.Map(
    location=wgs2gcj(116.552786, 39.766541)[::-1],
    control_scale=True,
    tiles='http://webrd02.is.autonavi.com/appmaptile?lang=zh_cn&size=1&scale=1&style=8&x={x}&y={y}&z={z}',
    attr='高德底图',
    zoom_start=13,
)
m.add_child(folium.LatLngPopup())
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
polys = []
for i, (name, poly) in enumerate(regions):
    color = colors[i % len(colors)]
    poly = poly[10:-2]
    points = [x.split(" ") for x in poly.split(", ")]
    if name == "京东贝":  # 可视化发现这个路区边界有问题, 特殊处理一下
        points = points[1:20]
        points.append(points[0])
    points = [gcj2wgs(float(lon), float(lat)) for lon, lat in points]
    assert points[0] == points[-1]
    polys.append(Polygon([projector(*p) for p in points]))
    folium.Polygon(
        locations=[wgs2gcj(*p)[::-1] for p in points],
        opacity=0.5,
        weight=3,
        popup=name,
        color=color,
        fill=True).add_to(m)
    centroid = Polygon(points).representative_point().coords[:][0]
    folium.CircleMarker(
        location=wgs2gcj(*centroid)[::-1],
        opacity=0.8,
        radius=3,
        fill=True,
        popup=name,
        color=color).add_to(m)

bound = unary_union([scale(p, xfact=1.01, yfact=1.01, origin='centroid') for p in polys])
assert isinstance(bound, Polygon)  # 所有路区的总边界
folium.PolyLine(
    locations=[wgs2gcj(*projector(*p, inverse=True))[::-1] for p in bound.exterior.coords[:]]
).add_to(m) 
m.save("figure/find_region_center.html")

In [ ]:
# LON_CEN, LAT_CEN
print(gcj2wgs(116.5622, 39.7874))

In [4]:
df = pd.read_csv(
    "orig_data/traj.csv", 
    usecols=["gps_time", "operatorid", "gps_lng", "gps_lat"], 
    dtype={"gps_time": str, "operatorid": int, "gps_lng": float, "gps_lat": float})
df = df[["operatorid", "gps_time", "gps_lng", "gps_lat"]]
df.head(5)
lines = df.values.tolist()

In [12]:
def tackle_multiple_coord(traj):
    """处理同时刻多个不同坐标的问题"""
    traj = list(set(traj))
    t2xys = defaultdict(list)
    for x, y, t in traj:
        t2xys[t].append((x, y))
    t_xys = list(t2xys.items())
    t_xys.sort(key=lambda x: x[0])
    traj = []
    multiple_cnt= 0
    for t, xys in t_xys:
        if len(xys) == 1:
            traj.append((*xys[0], float(t)))
        else:
            xs, ys = zip(*xys)
            x, y = np.mean(xs), np.mean(ys)  # 取平均坐标
            dis = np.mean([((x1 - x) ** 2 + (y1 - y) ** 2) ** 0.5 for x1, y1 in xys])
            if dis < 10:
                traj.append((x, y, float(t)))
            else:
                multiple_cnt += 1
    return traj, multiple_cnt

In [19]:
cid_date_to_traj = defaultdict(list)
for cid, t, lon, lat in tqdm(lines):
    assert isinstance(cid, int)
    assert isinstance(lon, float)
    assert isinstance(lat, float)
    date = t.split(' ')[0]
    t = time.mktime(time.strptime(t, "%Y-%m-%d %H:%M:%S")) - \
        time.mktime(time.strptime(date + " 00:00:00", "%Y-%m-%d %H:%M:%S"))
    assert 0 <= t < 86400 and isinstance(t, float)
    x, y = projector(*gcj2wgs(lon, lat))
    cid_date_to_traj[cid, date].append((x, y, t))
multiple_cnt = 0
traj_len = 0
for (cid, date), traj in cid_date_to_traj.items():
    traj, cnt = tackle_multiple_coord(traj)
    cid_date_to_traj[cid, date] = traj
    multiple_cnt += cnt
    traj_len += len(traj)
print(traj_len, multiple_cnt)

100%|██████████| 23506527/23506527 [24:33<00:00, 15951.05it/s]


22876681 243160


In [29]:
# 可视化快递员轨迹, 推断出仓库的位置, 作为constants_spatial.py中的LON_STA, LAT_STA
cid2trajs = defaultdict(list)
for (c, d), traj in cid_date_to_traj.items():
    if len(traj) > 0:
        cid2trajs[c].append(traj)
print(len(cid2trajs))

135


In [ ]:
# 快递员数量太多了, 推测是轨迹数据给多了; 看看订单数据里面有多少快递员
a = pd.read_csv("orig_data/order_2023-02-01_2023-02-28.csv")
b = pd.read_csv("orig_data/order_2023-03-01_2023-03-14.csv")
cid_set = set(a["operator_id"]) | set(b["operator_id"])
print(len(cid_set))
cid2trajs = {k: v for k, v in cid2trajs.items() if k in cid_set}
print(len(cid2trajs))

In [ ]:
# 顺手看一下快递员都叫啥名
a = pd.read_csv("orig_data/order_2023-02-01_2023-02-28.csv", usecols=["operator_id", "小哥姓名"])
b = pd.read_csv("orig_data/order_2023-03-01_2023-03-14.csv", usecols=["operator_id", "小哥姓名"])
cid2names = defaultdict(set)
for data in [a, b]:
    for cid, name in data.values:
        cid2names[int(cid)].add(name)
for v in cid2names.values():
    assert len(v) == 1
cid2names = {cid: list(names)[0] for cid, names in cid2names.items()}
pprint(cid2names)

In [34]:
m = folium.Map(
    location=wgs2gcj(116.552786, 39.766541)[::-1],
    control_scale=True,
    tiles='http://webrd02.is.autonavi.com/appmaptile?lang=zh_cn&size=1&scale=1&style=8&x={x}&y={y}&z={z}',
    attr='高德底图',
    zoom_start=13,
)
m.add_child(folium.LatLngPopup())
for i, (cid, trajs) in enumerate(cid2trajs.items()):
    color = colors[i % len(colors)]
    traj = max(trajs, key=lambda x: len(x))  # 每个小哥画轨迹最多的那天
    for x, y, t in traj[::10]:
        folium.CircleMarker(
            location=wgs2gcj(*projector(x, y, inverse=True))[::-1],
            radius=1,
            color=color,
            opcity=0.1).add_to(m)
folium.CircleMarker(
    location=[39.767614, 116.558602],  # 高德搜索京东北京鸿坤营业部的结果
    radius=10,
    color="red").add_to(m)
folium.PolyLine(
    locations=[wgs2gcj(*projector(*p, inverse=True))[::-1] for p in bound.exterior.coords[:]]
).add_to(m)
m.save(f"figure/find_station.html")

In [ ]:
print(gcj2wgs(116.558602, 39.767614))